## Семинар 7. Квантизация

В этом уроке мы детально разберем процесс квантизации, реализовав его с нуля. Затем мы добавим квантизацию в DistilBERT и измерим улучшения по скорости и используемой памяти.

### План

1. Датасет и модель
1. Реализация квантизации
1. Квантизация в torch
1. Квантизация в huggingface
1. PEFT для квантизованных моделей

## Данные

Будем работать с задачей классификации тональности на датасете IMDb.

In [6]:
from datasets import load_dataset

imdb = load_dataset("imdb", split='test')

/home/amshabalin/anaconda3/envs/karpov/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("imdb_distilbert_checkpoint")

/home/amshabalin/anaconda3/envs/karpov/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True, remove_columns=['text'])

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from torch.utils.data import DataLoader

dataloader = DataLoader(tokenized_imdb, collate_fn=data_collator, batch_size=64, shuffle=False)

In [11]:
batch = next(iter(dataloader))

## Модель

В качестве препарируемой модели мы возьмем дистиллированную версию BERT, `distilbert-base-uncased`. Для этого урока она была дообучена на IMDb в течение двух эпох.

In [9]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('imdb_distilbert_checkpoint', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [13]:
total_number_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_number_params

66955010

In [14]:
print('Модель занимает %.3f GB' % (model.get_memory_footprint() / 2**30))

Модель занимает 0.249 GB


Замеряем качество, чтобы было с чем сравниваться после квантизации.

In [20]:
from tqdm.auto import tqdm

@torch.no_grad()
def evaluate(model):
    model.eval()

    correct = 0
    for batch in tqdm(dataloader):
        output = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda()).logits.cpu()
        correct += (batch['labels'] == output.argmax(-1)).float().sum()

    accuracy = correct / len(dataloader.dataset)
    return accuracy

In [13]:
evaluate(model)

100%|██████████| 391/391 [02:04<00:00,  3.14it/s]


tensor(0.9313)

## Квантизация

Напомним, что квантизация в int производится по следующим формулам.

$$
x = s(x_q - z) \qquad x_q = \text{round}\bigg(\frac{x}{s} + z\bigg)
$$

В там случае можем вывести квантизованное умножение матриц для линейного слоя. Заметьте, что под знаком суммы стоят только квантизованные числа, для которых все операции быстрее.

\begin{aligned}
& Y_{i, j} \\
& = b_j+\sum_{k=1}^d X_{i, k} W_{k, j} \\
& = s_b\left(b_{q, j}-z_b\right)+\sum_{k=1}^d s_X\left(X_{q, i, k}-z_X\right) s_W\left(W_{q, k, j}-z_W\right) \\
& = s_b\left(b_{q, j}-z_b\right)+s_X s_W \sum_{k=1}^d\left(X_{q, i, k}-z_X\right)\left(W_{q, k, j}-z_W\right) \\
& = s_b\left(b_{q, j}-z_b\right)+s_X s_W\left[\left(\sum_{k=1}^d X_{q, i, k} W_{q, k, j}\right)-\left(z_W \sum_{k=1}^d X_{q, i, k}\right)-\left(z_X \sum_{k=1}^d W_{q, k, j}\right)+ d z_X z_W\right] \\
& = s_Y\left(Y_{q, i, j}-z_Y\right)
\end{aligned}

Осталось выразить отсюда $Y_{q, i, j}$.
\begin{aligned}
Y_{q, i, j} = round\Bigg( z_Y & +\frac{s_b}{s_Y}\left(b_{q, j}-z_b\right) \\
& +\frac{s_X s_W}{s_Y}\left[\left(\sum_{k=1}^d X_{q, i, k} W_{q, k, j}\right)-\left(z_W \sum_{k=1}^d X_{q, i, k}\right)-\left(z_X \sum_{k=1}^d W_{q, k, j}\right)+d z_X z_W\right] \Bigg)
\end{aligned}

Теперь реализуем такую квантизацию на `torch`.

In [16]:
from torch import nn

class QuantizedTensor(nn.Module):
    def __init__(self, x: torch.Tensor):
        super().__init__()
        
        self.a_q = -128
        self.b_q = 127

        # добавляем микро сдвиг, чтобы не делить на 0 в случае чего
        a, b = x.min() - 1e-6, x.max() + 1e-6
        self.s, self.z = self.quantization_constants(a, b)
        self.x_q = nn.Parameter(self.quantize(x).detach(), requires_grad=False)

    def quantize(self, x):
        x_q = torch.round(x / self.s + self.z)
        x_q = torch.clip(x_q, min=self.a_q, max=self.b_q)

        return x_q.to(torch.int8)

    def dequantize(self):
        x_q = self.x_q.to(torch.int32)
        x = self.s * (x_q - self.z)

        return x
    
    def quantization_constants(self, a, b):
        s = (b - a) / (self.b_q - self.a_q)
        z = int((b * self.a_q - a * self.b_q) / (b - a))

        return s, z

In [17]:
class QuantizedLinear(nn.Module):
    def __init__(self, module: nn.Linear):
        super().__init__()
        self.weight = QuantizedTensor(module.weight.data.T)
        if module.bias is not None:
            self.bias = QuantizedTensor(module.bias.data)
        else:
            self.bias = None

    def forward(self, x: torch.Tensor):
        d = self.weight.x_q.shape[-1]

        x = QuantizedTensor(x)

        if self.bias is not None:
            bias = self.bias.s * (self.bias.x_q - self.bias.z)
        else:
            bias = 0

        # переводим все в int32, чтобы не было переполнения
        product = (x.s * self.weight.s) * (
            torch.matmul(x.x_q.to(torch.int32), self.weight.x_q.to(torch.int32)) - \
            self.weight.z * torch.sum(x.x_q.to(torch.int32), axis=-1, keepdims=True) - \
            x.z * torch.sum(self.weight.x_q.to(torch.int32), axis=-2, keepdims=True) + \
            d * x.z * self.weight.z
        )
        return product + bias

### Тестируем

In [18]:
lin = nn.Linear(128, 128)
q_lin = QuantizedLinear(lin)

In [19]:
x = torch.rand(256, 128)

In [20]:
%%timeit

with torch.no_grad():
    lin(x)

51 μs ± 9.41 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [21]:
%%timeit

with torch.no_grad():
    q_lin(x)

1.23 ms ± 123 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


`torch` не оптимизирован под перемножение матриц с типом `int`, поэтому квантизованный слой работает медленнее.

In [22]:
expected_output = lin(x)
output = q_lin(x)

torch.abs(expected_output - output).mean().item()

0.0011311459820717573

Ошибка очень незначительна, значит мы реализовали все правильно.

In [27]:
def get_layer(model, name):
    layer = model
    for attr in name.split("."):
        layer = getattr(layer, attr)
    return layer

def set_layer(model, name, layer):
    if len(name.rsplit(".", 1)) > 1:
        attrs, name = name.rsplit(".", 1)
        model = get_layer(model, attrs)

    setattr(model, name, layer)

for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        module = QuantizedLinear(module)
        
        set_layer(model, name, module)

In [28]:
print('Модель занимает %.3f GB' % (model.get_memory_footprint() / 2**30))

Модель занимает 0.129 GB


С помощью такой квантизации мы уменьшили затраты по памяти в 2 раза. Могли бы больше, если бы квантизовали еще эмбеддинги, например.

## Torch квантизация

Для того, чтобы не реализовывать квантизацию самостоятельно, можно взять готовую из `torch`. В нем реализованы как динамическая, так и статическая квантизация, а так же различные методы калибровки. Однако у него есть и ряд недостатков, о них позже.

In [130]:
import torch

In [131]:
model = AutoModelForSequenceClassification.from_pretrained('imdb_distilbert_checkpoint', num_labels=2)

In [132]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [27]:
%%time

with torch.no_grad():
    output = model(batch['input_ids'], batch['attention_mask'])

CPU times: user 38.3 s, sys: 6.77 s, total: 45.1 s
Wall time: 24.5 s


In [27]:
model_int8 = torch.ao.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8  # специальный тип, который содержит еще коэффициенты квантизации
)
print('Модель занимает %.3f GB' % (model_int8.get_memory_footprint() / 2**30))

Модель занимает 0.089 GB


На самом деле больше, просто квантизованные веса не отображаются в списке параметров модели.

In [29]:
model_int8

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (k_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_lin): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (out_lin): DynamicQuantizedLinear(in_features=768, out_feature

In [30]:
for name, p in model_int8.named_parameters():
    print(name)

distilbert.embeddings.word_embeddings.weight
distilbert.embeddings.position_embeddings.weight
distilbert.embeddings.LayerNorm.weight
distilbert.embeddings.LayerNorm.bias
distilbert.transformer.layer.0.sa_layer_norm.weight
distilbert.transformer.layer.0.sa_layer_norm.bias
distilbert.transformer.layer.0.output_layer_norm.weight
distilbert.transformer.layer.0.output_layer_norm.bias
distilbert.transformer.layer.1.sa_layer_norm.weight
distilbert.transformer.layer.1.sa_layer_norm.bias
distilbert.transformer.layer.1.output_layer_norm.weight
distilbert.transformer.layer.1.output_layer_norm.bias
distilbert.transformer.layer.2.sa_layer_norm.weight
distilbert.transformer.layer.2.sa_layer_norm.bias
distilbert.transformer.layer.2.output_layer_norm.weight
distilbert.transformer.layer.2.output_layer_norm.bias
distilbert.transformer.layer.3.sa_layer_norm.weight
distilbert.transformer.layer.3.sa_layer_norm.bias
distilbert.transformer.layer.3.output_layer_norm.weight
distilbert.transformer.layer.3.outpu

Теперь замеряем скорость с квантизацией.

In [31]:
%%time

with torch.no_grad():
    output = model_int8(batch['input_ids'], batch['attention_mask'])

CPU times: user 33.1 s, sys: 6.84 s, total: 40 s
Wall time: 20.4 s


Видим, что стало быстрее, хоть и не очень сильно.

Мы запускам код на CPU, потому что в `torch` не поддерживается использование GPU с квантизованными параметрами.

In [28]:
model_int8.cuda();

try:
    with torch.no_grad():
        output = model_int8(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
except NotImplementedError as e:
    print(e)

Could not run 'quantized::linear_dynamic' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_dynamic' is only available for these backends: [CPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at ../aten/src/ATen/native/quantized/cpu/qlinear_dynamic.cpp:662 [kernel]
BackendSelect: fallthrough registered at

__Важно:__ Время работы квантизованных в `torch` моделей очень зависит от вашего железа и ОС. В некоторых случаях квантизация может значительно замедлять модель. Помимо этого, в разных версиях `torch` работа с квантизацией организована по-разному. В общем, лучше использовать другие библиотеки.

### BitsAndBytes

Методы квантизации из huggingface гораздо более приспособлены к практике. Достаточно просто указать конфиг и передать его при загрузке модели. Модель загрузится сразу на нужном устройстве с нужными типами весов.

In [10]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    'imdb_distilbert_checkpoint', quantization_config=quantization_config, num_labels=2
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [15]:
next(model.parameters()).device

device(type='cuda', index=0)

Все неквантизованные веса хранятся в fp16.

In [16]:
print('Модель занимает %.3f GB' % (model.get_memory_footprint() / 2**30))

Модель занимает 0.085 GB


In [17]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear8bitLt(in_features=768, out_features=768, bias=True)
            (k_lin): Linear8bitLt(in_features=768, out_features=768, bias=True)
            (v_lin): Linear8bitLt(in_features=768, out_features=768, bias=True)
            (out_lin): Linear8bitLt(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout

In [18]:
%%time

with torch.no_grad():
    output = model(batch['input_ids'], batch['attention_mask'])

CPU times: user 1.23 s, sys: 402 ms, total: 1.63 s
Wall time: 3.6 s


In [21]:
evaluate(model)

100%|██████████| 391/391 [03:50<00:00,  1.70it/s]


tensor(0.9266)

Такая квантизация ускорила применение модели больше чем в 1.5 раза без потери в качестве.

Помимо BitsAndBytes есть много других модулей для квантизации. Полный список можно посмотреть [тут](https://huggingface.co/docs/transformers/main/en/quantization/overview).

## Статическая квантизация

In [ ]:
from functools import partial
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoQuantizationConfig, AutoCalibrationConfig

model_id = 'distilbert/distilbert-base-uncased'

onnx_model = ORTModelForSequenceClassification.from_pretrained(model_id, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantizer = ORTQuantizer.from_pretrained(onnx_model)
qconfig = AutoQuantizationConfig.arm64(is_static=True, per_channel=False)

In [31]:
qconfig

QuantizationConfig(is_static=True, format=<QuantFormat.QDQ: 1>, mode=<QuantizationMode.QLinearOps: 1>, activations_dtype=<QuantType.QUInt8: 1>, activations_symmetric=False, weights_dtype=<QuantType.QInt8: 0>, weights_symmetric=True, per_channel=False, reduce_range=False, nodes_to_quantize=[], nodes_to_exclude=[], operators_to_quantize=['Conv', 'ConvTranspose', 'Gemm', 'Clip', 'Relu', 'Reshape', 'Transpose', 'Squeeze', 'Unsqueeze', 'Resize', 'MaxPool', 'AveragePool', 'MatMul', 'Split', 'Gather', 'Softmax', 'Where', 'InstanceNormalization'], qdq_add_pair_to_weight=False, qdq_dedicated_pair=False, qdq_op_type_per_channel_support_to_axis={'MatMul': 1})

In [33]:
calibration_dataset = tokenized_imdb.remove_columns("label").select(range(1000))

In [ ]:
calibration_config = AutoCalibrationConfig.minmax(calibration_dataset)

# калибруем диапазоны [a, b] для всех слоев
ranges = quantizer.fit(
    dataset=calibration_dataset,
    calibration_config=calibration_config,
    operators_to_quantize=qconfig.operators_to_quantize,
)

model_quantized_path = quantizer.quantize(
    save_dir="quantized_distilbert",
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
)

In [ ]:
onnx_path = 'onnx_path'

In [ ]:
from utils import create_quantization_preprocessor
 
# create processor
quantization_preprocessor = create_quantization_preprocessor()

# Quantize the same way we did for dynamic quantization!
quantizer.export(
    onnx_model_path=onnx_path / "model.onnx",
    onnx_quantized_model_output_path=onnx_path / "model-quantized.onnx",
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
    preprocessor=quantization_preprocessor,
)

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer
 
model = ORTModelForSequenceClassification.from_pretrained(onnx_path, file_name="model-quantized.onnx")

In [ ]:
%%time

with torch.no_grad():
    output = model(batch['input_ids'], batch['attention_mask'])

### Дообучение квантизованных моделей

Часто бывает полезно дообучать модели. Иногда эти модели настолько огромные, что помещаются в память только в квантизованном виде. В таком случае при дообучении тоже нужно использовать квантизацию. Все модули higgingface не поддерживают дообучение квантизованных весов из-за его нестабильности. Однако можно дообучить квантизованный адаптер, чаще всего используется [QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes).   
Ниже представлен пример, как это можно сделать.

In [1]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # квантизуем все веса в 4 бита
    bnb_4bit_quant_type="nf4",  # (4-bit NormalFloat) специальный тип переменных,
                                # который подходит для нормально распредененных весов
    bnb_4bit_use_double_quant=True,  # дополнительно квантизуем константы квантизации
    bnb_4bit_compute_dtype=torch.bfloat16,  # операции производятся в bfloat16 для ускорения
)

/home/amshabalin/anaconda3/envs/karpov/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import AutoModelForCausalLM

# Mistral тут для примера, может быть любая другая модель
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Вызываем `prepare_model_for_kbit_training` для модели. Это нужно для того, чтобы задать типы всех весов, в частности:
1. Перевести веса всех неквантизованных весов в fp32: Layer Norm, веса последнего линейного слоя.
2. Добавить `requires_grad=True` для эмбеддингов.

In [5]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig

config = LoraConfig(
    r=16,
    target_modules="all-linear",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

In [ ]:
quantized_model = get_peft_model(quantized_model, peft_config)

Дальше эту модель можно обучать через `Trainer` как обычно.